In [1]:
# the objective in this notebook is to take every single log in FlightLogs/ and turn them into data that can be examined
# individually (for cases like gusts of strong wind moving the drone around disrupting xSpeed and ySpeed data, for example)
# by dropping unnecessary columns, cleaning what's left, and splitting logs up based on 10m, 30m, and 50m status
# primarily tho, this is just cleanup, and everything before the function is just showing the thought process behind the cleaning
# decisions

In [10]:
# import tings
import pandas as pd
import numpy as np
from pathlib import Path

In [12]:
eg_log = pd.read_csv('FlightLogs/Jan9.12-44.csv')

In [13]:
# get started cleaning logs again to remove the stuff that doesn't matter
cols_to_drop = ['altitude_above_seaLevel(feet)', 
                     'altitude(feet)',
                     'flycStateRaw',
                     'height_sonar(feet)', 
                     'satellites', 
                     'max_altitude(feet)', 
                     'isPhoto', 
                     'isVideo',
                     'distance(feet)',
                     'mileage(feet)',
                     'max_ascent(feet)',
                     'max_distance(feet)',
                     'max_speed(mph)',
                     'gimbal_heading(degrees)',
                     'gimbal_pitch(degrees)',
                     'gimbal_roll(degrees)',
                     'voltageCell1',
                     'voltageCell2',
                     'voltageCell3',
                     'voltageCell4',
                     'voltageCell5',
                     'voltageCell6']
eg_log.drop(columns=cols_to_drop, inplace=True)

In [9]:
# some columns have a weird space in front of them, since every other columns has no space, we just split on spaces and keep what's left
eg_log.columns = eg_log.columns.str.lstrip()
eg_log.columns

Index(['time(millisecond)', 'timestamp', 'speed(mph)', 'gpslevel',
       'voltage(v)', 'xSpeed(mph)', 'ySpeed(mph)', 'zSpeed(mph)',
       'compass_heading(degrees)', 'pitch(degrees)', 'roll(degrees)',
       'rc_elevator', 'rc_aileron', 'rc_throttle', 'rc_rudder',
       'rc_elevator(percent)', 'rc_aileron(percent)', 'rc_throttle(percent)',
       'rc_rudder(percent)', 'battery_percent', 'current(A)',
       'battery_temperature(f)', 'ascent(feet)', 'flycState', 'message',
       'height_above_takeoff(meters)'],
      dtype='object')

In [6]:
# none of the data (besides the drone hovering at 10m, 30m, and 50m) matters. let's get keep only that
# and filter it down further later
eg_log_10m = eg_log[(eg_log['height_above_takeoff(meters)'] > 7) & (eg_log['height_above_takeoff(meters)'] < 13)]
eg_log_30m = eg_log[(eg_log['height_above_takeoff(meters)'] > 27) & (eg_log['height_above_takeoff(meters)'] < 33)]
eg_log_50m = eg_log[(eg_log['height_above_takeoff(meters)'] > 47) & (eg_log['height_above_takeoff(meters)'] < 53)]

In [7]:
# EXPLORATORY DATA PARSING DONE, NOW WE DO IT TO EVERY LOG WE HAVE

In [19]:
# set up paths to quickly make iterate over them all and make them all usable (usable-ify them)
directory = Path('FlightLogs')
preusable_logs = directory.glob('*.csv')
output_dir = Path('UsableLogs')

def usableify_the_logs():
    for log_file in preusable_logs:
        log = pd.read_csv(log_file)
        log.drop(columns=cols_to_drop, inplace=True)
        log.columns = log.columns.str.lstrip()

        # column 'rc_throttle(percent)' measures where I'm touching the controller or not, so we'll only take values
        # where it is 0. Also, since I don't touch the controls during AutoLanding, we'll check for 'flycState' being
        # P-GPS, which is the positioning mode.
        log = log[log['rc_throttle(percent)'] == 0]
        log = log[log['flycState'] == 'P-GPS']

        # the drone is still moving upward for about a second after releasing the sticks, so we'll remove the first 12 rows
        # (translates to about 1.2 seconds after releasing the sticks)
        # make the new _##m files to aggregate upon later
        log_10m = log[(log['height_above_takeoff(meters)'] > 7) & (log['height_above_takeoff(meters)'] < 13)].reset_index().loc[12:]
        log_30m = log[(log['height_above_takeoff(meters)'] > 27) & (log['height_above_takeoff(meters)'] < 33)].reset_index().loc[12:]
        log_50m = log[(log['height_above_takeoff(meters)'] > 47) & (log['height_above_takeoff(meters)'] < 53)].reset_index().loc[12:]

        # push em all to csv in a new 'usable' folder
        log_10m.to_csv(output_dir / f"{log_file.stem}_10m.csv", index=False) # .stem gets the file name without the .csv part
        log_30m.to_csv(output_dir / f"{log_file.stem}_30m.csv", index=False)
        log_50m.to_csv(output_dir / f"{log_file.stem}_50m.csv", index=False)

In [20]:
# run the function
usableify_the_logs()